# 傾向スコアマッチング

In [12]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from psmpy import PsmPy
from psmpy.functions import cohenD

In [6]:
filepath = '../../../data/processed/imputed_df.xlsx'
df = pd.read_excel(filepath)
df['log_income'] = np.log(df['income'])
df.head()

,year,island_id,population,dummy_after_bridge_opened,income,log_income
0,1995,0,15415.5,0,16109099.0,16.594895
1,1995,1,2857.0,0,2443874.0,14.709095
2,1995,2,4048.0,0,3451278.0,15.054255
3,1995,3,802.0,0,535779.0,13.191477
4,1995,4,4597.0,0,4017388.0,15.206142


## 処置変数と共変量の指定

In [7]:
treatment = df['dummy_after_bridge_opened']
covariates = df[['population', 'log_income']]

## ロジスティック回帰モデルの適用

In [15]:
logistic = smf.logit('dummy_after_bridge_opened ~ population + log_income + C(island_id) + C(year)', data=df).fit()

/home/keisei/data-analysis/bridge-straw-effects/.venv/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


         Current function value: inf
         Iterations: 35


/home/keisei/data-analysis/bridge-straw-effects/.venv/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:2443: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))


LinAlgError: Singular matrix

## 傾向スコアの推定

In [10]:
df['propensity_score'] = logistic.predict_proba(covariates)[:, 1]

In [11]:
df

,year,island_id,population,dummy_after_bridge_opened,income,log_income,propensity_score
0,1995,0,15415.5,0,16109099.0,16.594895,0.738771
1,1995,1,2857.0,0,2443874.0,14.709095,0.282189
2,1995,2,4048.0,0,3451278.0,15.054255,0.323180
3,1995,3,802.0,0,535779.0,13.191477,0.212557
4,1995,4,4597.0,0,4017388.0,15.206142,0.343014
...,...,...,...,...,...,...,...
919,2020,158,57.0,1,20918415.0,16.856140,0.220821
920,2020,160,225.0,1,3895125.0,15.175236,0.212592
921,2020,161,429.0,1,20781354.0,16.849567,0.230555
922,2020,162,38.0,0,20781354.0,16.849567,0.220280
